In [1]:
from util.visualise import *

In [2]:
PAPER_DIR = str(Path.home())+"/papers/24-aaai-goose/paper"

PLOT_DIR = f"{PAPER_DIR}/plots"
PLOT_DIR

'/home/dillon/papers/24-aaai-goose/paper/plots'

In [3]:
L=4
aggr="max"
H=64

CONFIGS = ["hff", "sdg-el dd", "fdg-el dd", "ldg-el dd", "sdg-el di", "fdg-el di", "ldg-el di"]

In [49]:
h_opt = {}
for domain in GOOSE_DOMAINS:
    problem_pddls = sorted_nicely(os.listdir(f"../benchmarks/goose/{domain}/test"))
    for problem_pddl in problem_pddls:
        problem_name = os.path.basename(problem_pddl).replace(".pddl", "")

        key = (domain, problem_name)
        f = f'logs/seq-opt-mns/{domain}_{problem_name}_seq-opt-mns.log'
        for line in open(f, 'r').readlines():
            if 'Plan cost: ' in line:
                h_opt[key] = int(line.split()[-1])
        
        if key in h_opt:
            continue

        # hardcode opt for some problems
        if domain == "gripper":
            n = int(problem_name.replace("gripper-n", ""))
            h_opt[key] = n * 3 - (n % 2 == 0)
        elif domain == "visitall":
            for i in range(5):
                problem_name = problem_name.replace(f"-s{i}", '')
            n = int(problem_name.replace("n", ""))
            h_opt[key] = n * n - 1
        elif domain == "spanner":
            nums = [int(s) for s in re.findall(r'\d+', problem_name)]
            s = nums[0]
            n = nums[1]
            l = nums[2]
            h_opt[key] = 2*n + l + 1
        elif domain == "ferry":
            nums = [int(s) for s in re.findall(r'\d+', problem_name)]
            n_locs = nums[0]
            n_cars = nums[1]
            ferry = -1

            locs = {l:set() for l in range(n_locs)}  # only stores non goal cars
            cars = {}
            goal = {}

            goal_time = False
            for line in open(f"../benchmarks/goose/{domain}/test/{problem_pddl}", 'r').readlines():
                if "(:goal" in line:
                    goal_time = True
                    continue

                if "(at-ferry" in line:
                    line = line.split()
                    ferry = int(line[-1].replace('l', '').replace(')', ''))
                    continue

                if "(at " in line:
                    line = line.split()
                    car = int(line[1].replace('c', ''))
                    loc = int(line[-1].replace('l', '').replace(')', ''))
                    if not goal_time:
                        locs[loc].add(car)
                        cars[car] = loc
                    else:
                        goal[car] = loc
            
            # solve problem
            opt = 0
            while True:
                # check goal:
                is_goal = True
                for car in range(n_cars):
                    if cars[car] != goal[car]:
                        is_goal = False
                        break
                if is_goal:
                    break
                    
                # greedy find car whose goal location has non goal cars
                car_to_transport = -1
                for car in locs[ferry]:
                    if cars[car] != goal[car] and len(locs[goal[car]]) > 0:
                        car_to_transport = car
                        car_loc = ferry
                        break
                if car_to_transport == -1:
                    # if all car goal locations are empty
                    for car in locs[ferry]:
                        if cars[car] != goal[car]:
                            car_to_transport = car
                            car_loc = ferry
                            break
                
                if car_to_transport != -1:
                    locs[car_loc].remove(car)
                    ferry = goal[car]
                    cars[car_to_transport] = goal[car_to_transport]
                    opt += 3
                else:
                    for car in range(n_cars):
                        if cars[car] != goal[car]:
                            car_to_transport = car
                            car_loc = cars[car]
                            break

                    locs[car_loc].remove(car)
                    ferry = goal[car]
                    cars[car_to_transport] = goal[car_to_transport]
                    opt += 4
            # end while
            h_opt[key] = opt

In [50]:
def scatter_plot(config):
    biggest_x = 0
    for domain in GOOSE_DOMAINS:
        xs = []
        ys = []

        problem_pddls = os.listdir(f"../benchmarks/goose/{domain}/test")
        for problem_pddl in problem_pddls:
            problem_name = os.path.basename(problem_pddl).replace(".pddl", "")

            key = (domain, problem_name)
            if key not in h_opt:
                continue
            
            config_toks = config.split()
            if len(config_toks) == 2:
                rep, train_type = tuple(config_toks)
                p = 20 if train_type == "di" else 10
                f = f'logs/test/{problem_name}_{train_type}_{rep}_{domain}_L{L}_H{H}_{aggr}_p{p}_r0.log'
                if not os.path.exists(f): continue
            else:  #hff
                f = f'logs/{config}/{domain}_{problem_name}_{config}.log'

            
            x = h_opt[key]
            y = scrape_search_log(f)["first_h"]

            xs.append(x)
            ys.append(y)

            biggest_x = max(biggest_x, x)
        
        plt.scatter(xs, ys, label=domain, marker='x', alpha=0.8)
    plt.plot([1, biggest_x], [1, biggest_x], linestyle='dashed', color='k', zorder=0)
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(left=10)
    plt.ylim(bottom=10)
    plt.legend()
    # plt.show()
    plt.tight_layout()
    plt.savefig(f"{PLOT_DIR}/heuristic-{config.replace(' ', '_')}.pdf")
    plt.clf()
    return

In [51]:
for config in CONFIGS:
    scatter_plot(config)

<Figure size 432x288 with 0 Axes>